<br><br><br><br>
# Set up the environment

In [1]:
%load_ext sql

In [2]:
%%sql postgresql://postgres@localhost/postgres

select 'hi there' as "message";

1 rows affected.


message
hi there


<br><br><br><br>
# Define the schema and data

We define a `transactions` table to track financial transactions and a history table for capturing all changes made.

In [3]:
%%sql

drop table if exists transactions;

create table transactions (
    transaction_id serial primary key,
    user_id        text,
    merchant       text,
    amount         numeric,
    when_created   timestamp
);

drop table if exists transactions_history;

create table transactions_history (
    operation char(6) not null,
    when_modified  timestamp not null,
    transaction_id int,
    user_id        text,
    merchant       text,
    amount         numeric,
    when_created   timestamp
);

 * postgresql://postgres@localhost/postgres
Done.
Done.
Done.
Done.


[]

Now, we define a PostgreSQL trigger to populate the history table. Don't worry about the details here; they're beyond the scope of this talk.

In [4]:
%%sql

CREATE OR REPLACE FUNCTION capture_transactions_history() RETURNS TRIGGER AS $transactions_history$
    BEGIN
        IF (TG_OP = 'DELETE') THEN
            INSERT INTO transactions_history SELECT 'delete', now(), OLD.*;
            RETURN OLD;
        ELSIF (TG_OP = 'UPDATE') THEN
            INSERT INTO transactions_history SELECT 'update', now(), NEW.*;
            RETURN NEW;
        ELSIF (TG_OP = 'INSERT') THEN
            INSERT INTO transactions_history SELECT 'insert', now(), NEW.*;
            RETURN NEW;
        END IF;
        RETURN NULL; -- result is ignored since this is an AFTER trigger
    END;
$transactions_history$ LANGUAGE plpgsql;

DROP TRIGGER IF EXISTS transactions_history ON users;
CREATE TRIGGER transactions_history
AFTER INSERT OR UPDATE OR DELETE ON transactions
    FOR EACH ROW EXECUTE PROCEDURE capture_transactions_history();

 * postgresql://postgres@localhost/postgres
Done.
Done.
Done.


[]

Let's populate our table with data for two customers:

In [5]:
%%sql

delete from transactions;
insert into transactions (user_id, merchant, amount, when_created) values 
  ('jane', 'BP', 43.22, '2018-01-01 10:00'),
  ('jane', 'Magic Mountain', 24.11, '2018-01-01 14:30'),
  ('jane', 'Target', 2.00, '2018-01-01 10:30'),
  ('jill', 'Starbucks', 4.39, '2018-01-03 06:11'),
  ('jill', 'COTA', 20.00, '2018-01-03 06:15'),
  ('jill', 'Jeni Ice Cream', 7.23, '2018-01-04 15:12');

 * postgresql://postgres@localhost/postgres
0 rows affected.
6 rows affected.


[]

Let's see how the table looks:

In [6]:
%%sql

select * from transactions;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created
1,jane,BP,43.22,2018-01-01 10:00:00
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
3,jane,Target,2.00,2018-01-01 10:30:00
4,jill,Starbucks,4.39,2018-01-03 06:11:00
5,jill,COTA,20.00,2018-01-03 06:15:00
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00


And the history table looks very similar at this point:

In [7]:
%%sql

select * from transactions_history;

 * postgresql://postgres@localhost/postgres
6 rows affected.


operation,when_modified,transaction_id,user_id,merchant,amount,when_created
insert,2018-10-25 16:52:28.392892,1,jane,BP,43.22,2018-01-01 10:00:00
insert,2018-10-25 16:52:28.392892,2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
insert,2018-10-25 16:52:28.392892,3,jane,Target,2.00,2018-01-01 10:30:00
insert,2018-10-25 16:52:28.392892,4,jill,Starbucks,4.39,2018-01-03 06:11:00
insert,2018-10-25 16:52:28.392892,5,jill,COTA,20.00,2018-01-03 06:15:00
insert,2018-10-25 16:52:28.392892,6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00


Now that we have a dataset, let's review the concepts around SQL aggregate functions and run some more interesting queries.

<br/><br/><br/><br/>

# Aggregate functions (SUM, COUNT, etc.)

We can use an aggregate function like SUM to aggregate over many rows, returning a single row defining the results. For example:

In [8]:
%%sql

select sum(amount) as all_transactions_sum 
from transactions;

 * postgresql://postgres@localhost/postgres
1 rows affected.


all_transactions_sum
100.95


We can use GROUP BY to return multiple rows, each corresponding to a subset of the data:

In [9]:
%%sql

select user_id, sum(amount) as per_user_sum
from transactions 
group by user_id;

 * postgresql://postgres@localhost/postgres
2 rows affected.


user_id,per_user_sum
jill,31.62
jane,69.33


And we can do more complex transformations by adding additional columns and ordering expressions:

In [10]:
%%sql

select user_id, when_created::date, sum(amount) as per_user_day_sum
from transactions 
group by 1, 2
order by 1, 2;

 * postgresql://postgres@localhost/postgres
3 rows affected.


user_id,when_created,per_user_day_sum
jane,2018-01-01,69.33
jill,2018-01-03,24.39
jill,2018-01-04,7.23


<br><br><br><br>
# Window functions

Window functions are related to aggregations, but they don't "collapse" rows in the same way. How does this compare to the non-windowed select sum(amount) from transactions we saw earlier?

In [11]:
%%sql

select sum(amount) over () as all_transactions_sum
from transactions;

 * postgresql://postgres@localhost/postgres
6 rows affected.


all_transactions_sum
100.95
100.95
100.95
100.95
100.95
100.95


Window functions allow us to put aggregates right alongside the contents of the original rows.

In [12]:
%%sql

select *, sum(amount) over () as all_transactions_sum
from transactions;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created,all_transactions_sum
1,jane,BP,43.22,2018-01-01 10:00:00,100.95
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00,100.95
3,jane,Target,2.00,2018-01-01 10:30:00,100.95
4,jill,Starbucks,4.39,2018-01-03 06:11:00,100.95
5,jill,COTA,20.00,2018-01-03 06:15:00,100.95
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00,100.95


We can define windows over portions of the data, similar to GROUP BY for aggregates. But for window functions, these are called _partitions_:

In [13]:
%%sql

select 
*, 
sum(amount) over () as all_transactions_sum,
sum(amount) over (partition by user_id) as user_sum
from transactions
order by transaction_id;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created,all_transactions_sum,user_sum
1,jane,BP,43.22,2018-01-01 10:00:00,100.95,69.33
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00,100.95,69.33
3,jane,Target,2.00,2018-01-01 10:30:00,100.95,69.33
4,jill,Starbucks,4.39,2018-01-03 06:11:00,100.95,31.62
5,jill,COTA,20.00,2018-01-03 06:15:00,100.95,31.62
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00,100.95,31.62


We can further narrow the window by specifying a _frame_. Implicit in the above query is the default frame, which considers the entire partition. Making the frame explicit looks like:

In [14]:
%%sql

select *, 
sum(amount) over (
    partition by user_id
    rows between unbounded preceding and current row
)
from transactions
order by transaction_id;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created,sum
1,jane,BP,43.22,2018-01-01 10:00:00,43.22
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00,67.33
3,jane,Target,2.00,2018-01-01 10:30:00,69.33
4,jill,Starbucks,4.39,2018-01-03 06:11:00,4.39
5,jill,COTA,20.00,2018-01-03 06:15:00,24.39
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00,31.62


We can also specify ordering within the partition using ORDER BY, same as aggregations:

In [15]:
%%sql

select *, 
sum(amount) over (
    partition by user_id
    order by when_created
    rows between unbounded preceding and current row
)
from transactions;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created,sum
1,jane,BP,43.22,2018-01-01 10:00:00,43.22
3,jane,Target,2.00,2018-01-01 10:30:00,45.22
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00,69.33
4,jill,Starbucks,4.39,2018-01-03 06:11:00,4.39
5,jill,COTA,20.00,2018-01-03 06:15:00,24.39
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00,31.62


<br><br><br><br>
# History tables (aka audit tables)

Right now our transactions table and the history table contain equivalent rows because we've only had inserts:

In [16]:
%%sql

select * from transactions
order by transaction_id;

 * postgresql://postgres@localhost/postgres
6 rows affected.


transaction_id,user_id,merchant,amount,when_created
1,jane,BP,43.22,2018-01-01 10:00:00
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
3,jane,Target,2.00,2018-01-01 10:30:00
4,jill,Starbucks,4.39,2018-01-03 06:11:00
5,jill,COTA,20.00,2018-01-03 06:15:00
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00


In [17]:
%%sql

select * from transactions_history;

 * postgresql://postgres@localhost/postgres
6 rows affected.


operation,when_modified,transaction_id,user_id,merchant,amount,when_created
insert,2018-10-25 16:52:28.392892,1,jane,BP,43.22,2018-01-01 10:00:00
insert,2018-10-25 16:52:28.392892,2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
insert,2018-10-25 16:52:28.392892,3,jane,Target,2.00,2018-01-01 10:30:00
insert,2018-10-25 16:52:28.392892,4,jill,Starbucks,4.39,2018-01-03 06:11:00
insert,2018-10-25 16:52:28.392892,5,jill,COTA,20.00,2018-01-03 06:15:00
insert,2018-10-25 16:52:28.392892,6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00


Let's mutate the table and then check how the tables change.

In [18]:
%%sql

update transactions set amount = 5.39 where transaction_id = 4;
update transactions set amount = 3.39 where transaction_id = 4;
delete from transactions where transaction_id = 5;

 * postgresql://postgres@localhost/postgres
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [21]:
%%sql

select * from transactions
order by transaction_id;

 * postgresql://postgres@localhost/postgres
5 rows affected.


transaction_id,user_id,merchant,amount,when_created
1,jane,BP,43.22,2018-01-01 10:00:00
2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
3,jane,Target,2.00,2018-01-01 10:30:00
4,jill,Starbucks,3.39,2018-01-03 06:11:00
6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00


In [22]:
%%sql

select * from transactions_history;

 * postgresql://postgres@localhost/postgres
9 rows affected.


operation,when_modified,transaction_id,user_id,merchant,amount,when_created
insert,2018-10-25 16:52:28.392892,1,jane,BP,43.22,2018-01-01 10:00:00
insert,2018-10-25 16:52:28.392892,2,jane,Magic Mountain,24.11,2018-01-01 14:30:00
insert,2018-10-25 16:52:28.392892,3,jane,Target,2.00,2018-01-01 10:30:00
insert,2018-10-25 16:52:28.392892,4,jill,Starbucks,4.39,2018-01-03 06:11:00
insert,2018-10-25 16:52:28.392892,5,jill,COTA,20.00,2018-01-03 06:15:00
insert,2018-10-25 16:52:28.392892,6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00
update,2018-10-25 16:55:41.376200,4,jill,Starbucks,5.39,2018-01-03 06:11:00
update,2018-10-25 16:55:41.384644,4,jill,Starbucks,3.39,2018-01-03 06:11:00
delete,2018-10-25 16:55:41.388490,5,jill,COTA,20.00,2018-01-03 06:15:00


<br><br><br><br>
# Recreating current state from the history table

Now, imagine that we don't have access to the `transactions` table. Let's think through together how we could use our new knowledge of window functions to construct a query over `transactions_history` that recreates the current state of `transactions`.

In [23]:
%%sql

select * from
(
    select *,
    row_number() over (
        partition by transaction_id 
        order by when_modified desc
    ) as n
    from transactions_history
) as subquery
where n = 1
and operation != 'delete'

 * postgresql://postgres@localhost/postgres
5 rows affected.


operation,when_modified,transaction_id,user_id,merchant,amount,when_created,n
insert,2018-10-25 16:52:28.392892,1,jane,BP,43.22,2018-01-01 10:00:00,1
insert,2018-10-25 16:52:28.392892,2,jane,Magic Mountain,24.11,2018-01-01 14:30:00,1
insert,2018-10-25 16:52:28.392892,3,jane,Target,2.00,2018-01-01 10:30:00,1
update,2018-10-25 16:55:41.384644,4,jill,Starbucks,3.39,2018-01-03 06:11:00,1
insert,2018-10-25 16:52:28.392892,6,jill,Jeni Ice Cream,7.23,2018-01-04 15:12:00,1


<br><br><br><br>
# Dumping Postgres schema and data

Export the transactions_history table using PostgreSQL's `pg_dump` shell utility. We will be able to replay this into Redshift.

```
PGUSER=postgres PGPASSWORD=mysecretpassword pg_dump \
  --host localhost
  --port 5432
  --dbname postgres \
  --table transactions_history \
  --column-inserts \
  > dumpfile.sql
```